# get_frequently_mutated Final 

In [1]:
import cptac
#en = cptac.Endometrial()
#cptac.sync('endometrial', version='2.1')
#ov = cptac.Ovarian()
#cptac.sync(dataset='ovarian', version='0.0')
#cptac.sync(dataset='colon', version='0.0')
co = cptac.Colon()

import pandas as pd
import numpy as np

In [2]:
def get_frequently_mutated(cancer_object, cutoff = 0.1):  
    """
    take cancer object and find the frequently 
    mutated genes in the total tumors compared to the cutoff.

    Parameters:
    cancer_object (object): cancer class object from cptac module 
    cutoff (float): used as comparison to determine the 
                    status of gene mutation frequency

    Returns:
    freq_mutated_df (pd.DataFrame): DataFrame of frequently 
        mutated genes passing the cutoff. Columns contain the 
        fractions of total unique mutations,missence type 
        mutations, and truncation type mutations per gene.
    
    The Missence_Mut column includes: 
        In_Frame_Del, In_Frame_Ins, Missense_Mutation
   
   The Truncation_Mut column includes: 
        Frame_Shift_Del, Frame_Shift_Ins, Splice_Site, 
        Nonsense_Mutation, Nonstop_Mutation
        
    These columns count multiple mutations of one gene in the 
    same sample, so fractions in the last two columns may 
    exceed the Unique_Samples_Mut column which only counts if 
    the gene was mutated once per sample."""    
    
    # Get mutations data frame (whole exome sequencing)
    somatic_mutations = cancer_object.get_mutations()
    sample_status_map = cancer_object._get_sample_status_map()
         
    # Get total tumors/patients
    sample_status_count = sample_status_map.value_counts().reset_index()
    tumors = sample_status_count.loc[sample_status_count['index'] == 'Tumor']
    total_tumor_patients = tumors.iloc[0,1] #number of tumors 
    print(total_tumor_patients)
    # Find frequently mutated genes and their fraction of unique mutated samples.
    #move 'Sample_ID' from index to column
    origin_df = somatic_mutations.reset_index()
    
    # Drop silent mutations for Ovarian dataset
    if cancer_object.get_cancer_type() == 'ovarian':
        origin_df = origin_df.loc[origin_df['Mutation'] != 'Silent']
        #print(origin_df['Mutation'].unique())
        
    #group by gene and count unique samples
    count_mutations = origin_df.groupby(['Gene']).nunique()

    #format
    count_mutations = count_mutations.rename(columns={"Sample_ID": "Unique_Samples_Mut"})
    count_mutations = count_mutations.drop(['Gene', 'Mutation','Location'], axis = 1)
    
    #filter using the cutoff and create fraction
    fraction_mutated = count_mutations.apply(lambda x: x / total_tumor_patients)
    fraction_greater_than_cutoff = fraction_mutated.where(lambda x: x > cutoff) #na used when false
    filtered_gene_df = fraction_greater_than_cutoff.dropna()
    
    
    #Create Missence and Trucation data frame
    
    #create two categories in Mutation column
    if cancer_object.get_cancer_type() == 'colon':
        missence_truncation_groups = {'frameshift substitution': 'T', 
            'frameshift deletion': 'T', 'frameshift insertion': 'T', 
            'stopgain': 'T', 'stoploss': 'T', 'nonsynonymous SNV': 'M',
            'nonframeshift insertion': 'M','nonframeshift deletion': 'M', 
            'nonframeshift substitution': 'M'}
    else: 
        missence_truncation_groups = {'In_Frame_Del': 'M', 'In_Frame_Ins': 'M',
            'Missense_Mutation': 'M', 'Frame_Shift_Del': 'T','Nonsense_Mutation': 'T', 
            'Splice_Site': 'T', 'Frame_Shift_Ins': 'T','Nonstop_Mutation':'T'}
       
    mutations_replaced_M_T = origin_df.replace(missence_truncation_groups)
    
    # group mutation categories
    miss = mutations_replaced_M_T.loc[mutations_replaced_M_T['Mutation'] == 'M']
    trunc = mutations_replaced_M_T.loc[mutations_replaced_M_T['Mutation'] == 'T']

    # group by gene and count unique samples for both categories
    count_miss = miss.groupby(['Gene']).nunique()
    count_trunc = trunc.groupby(['Gene']).nunique()

    #format
    missence_df = count_miss.rename(columns={"Sample_ID": "Missence_Mut"})
    missence_df = missence_df.drop(['Gene', 'Mutation', 'Location'], axis = 1)

    truncation_df = count_trunc.rename(columns={"Sample_ID": "Truncation_Mut"})
    truncation_df = truncation_df.drop(['Gene', 'Mutation', 'Location'], axis = 1)

    #join miss and trunc and change nan to 0, then divide by total tumors
    join_mutations = missence_df.join(truncation_df).fillna(0)
    missence_and_truncation_df = join_mutations.apply(lambda x: x / total_tumor_patients)


    #Join data frames, keeping only the genes that passed the cutoff 
    freq_mutated_df = filtered_gene_df.join(missence_and_truncation_df).reset_index()
    freq_mutated_df.name = 'frequently_mutated'
    
    return freq_mutated_df

Test Colon

In [3]:
df = get_frequently_mutated(co, .1)

110


In [4]:
df

,Gene,Unique_Samples_Mut,Missence_Mut,Truncation_Mut
0,ABCA13,0.172727,0.145455,0.090909
1,ABCA2,0.154545,0.145455,0.027273
2,ABCA4,0.127273,0.072727,0.054545
3,ABCB4,0.118182,0.054545,0.063636
4,ACACB,0.109091,0.100000,0.009091
5,ACVR2A,0.227273,0.018182,0.227273
6,ADAMTS16,0.127273,0.118182,0.009091
7,ADAMTS5,0.109091,0.100000,0.018182
8,ADAMTS7,0.109091,0.109091,0.000000
9,ADAMTSL4,0.109091,0.054545,0.063636


In [58]:
#test results
m = ov.get_mutations()
gene = 'TTN'
g = m.loc[m['Gene'] == gene]
print(g)
print(len(g.index.unique()))

r = g.groupby('Mutation')

r.groups


          Gene           Mutation         Location
Sample_ID                                         
S002       TTN  Missense_Mutation        p.P25604T
S009       TTN        Splice_Site   p.X8828_splice
S009       TTN  Missense_Mutation         p.M2411K
S013       TTN             Silent        p.S14806S
S013       TTN  Nonsense_Mutation         p.R7040*
S015       TTN  Missense_Mutation        p.P27473S
S022       TTN    Frame_Shift_Ins    p.N25635Kfs*5
S028       TTN             Silent         p.K6047K
S029       TTN             Silent         p.V8411V
S037       TTN  Missense_Mutation        p.F31025L
S038       TTN  Missense_Mutation        p.S31594R
S038       TTN    Frame_Shift_Del    p.S22176Lfs*7
S038       TTN  Missense_Mutation        p.A12166P
S041       TTN             Silent         p.A8541A
S046       TTN  Missense_Mutation        p.L14165V
S046       TTN  Missense_Mutation         p.R9840C
S047       TTN  Missense_Mutation          p.E465Q
S051       TTN  Missense_Mutati

{'Frame_Shift_Del': Index(['S038', 'S071'], dtype='object', name='Sample_ID'),
 'Frame_Shift_Ins': Index(['S022'], dtype='object', name='Sample_ID'),
 'Missense_Mutation': Index(['S002', 'S009', 'S015', 'S037', 'S038', 'S038', 'S046', 'S046', 'S047',
        'S051', 'S054', 'S054', 'S056', 'S062', 'S066', 'S066', 'S071', 'S078',
        'S080', 'S080', 'S080', 'S080', 'S080', 'S080', 'S080', 'S080', 'S080',
        'S080', 'S080', 'S080', 'S080', 'S080', 'S080', 'S080', 'S080', 'S080',
        'S080', 'S080', 'S080', 'S080', 'S080', 'S080', 'S080', 'S084', 'S096',
        'S097', 'S097', 'S100', 'S101', 'S108', 'S108', 'S110'],
       dtype='object', name='Sample_ID'),
 'Nonsense_Mutation': Index(['S013', 'S080', 'S080', 'S080', 'S080', 'S080'], dtype='object', name='Sample_ID'),
 'Silent': Index(['S013', 'S028', 'S029', 'S041', 'S054', 'S071', 'S074', 'S078', 'S080',
        'S080', 'S080', 'S080', 'S080', 'S080', 'S090', 'S095'],
       dtype='object', name='Sample_ID'),
 'Splice_Sit

Test Colon

In [ ]:
colon_freq_mutated_df = get_frequently_mutated(cancer, .25)

In [ ]:
colon_freq_mutated_df

Test Ovarian

In [ ]:
ovarian_freq_mutated_df = get_frequently_mutated(ov,.1)

In [ ]:
ovarian_freq_mutated_df